#import the important libraries

In [37]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
from pandas.io.json import json_normalize  # tranform JSON file into a pandas dataframe

import folium # map rendering library

# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

 In order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [38]:
toronto_df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Ignore cells with a borough that is Not assigned. 

In [44]:
#Filtering out Boroughs which are Not assigned
toronto_df = toronto_df[toronto_df.Borough!="Not assigned"]
toronto_df.head()

,Postal Code,Borough,Neighbourhood,new_nghbr
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"


In [45]:
toronto_dropNotAssigned = toronto_df[toronto_df.Borough != 'Not assigned'].reset_index(drop=True)
toronto_df.head()

,Postal Code,Borough,Neighbourhood,new_nghbr
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"


Group all neighborhoods with the same postal code

In [62]:
toronto_df = toronto_df.groupby(["Postal Code", "Borough"])["Neighbourhood"].apply(", ".join).reset_index()
toronto_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [46]:
toronto_df['new_nghbr'] = np.where(toronto_df['Neighbourhood']=='Not assigned',toronto_df['Borough'],toronto_df['Neighbourhood'])
toronto_df.head(10)

,Postal Code,Borough,Neighbourhood,new_nghbr
2,M3A,North York,Parkwoods,Parkwoods
3,M4A,North York,Victoria Village,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront","Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights","Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government","Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village","Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge","Malvern, Rouge"
11,M3B,North York,Don Mills,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens","Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson","Garden District, Ryerson"


In [68]:
toronto_df.head()

,Postal Code,Borough,Neighbourhood,new_nghbr
0,M1A,Not assigned,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge","Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill","Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn,Woburn


In [65]:
toronto_df['new_nghbr'] = np.where(toronto_df['Neighbourhood']=='Not assigned',toronto_df['Borough'],toronto_df['Neighbourhood'])
toronto_df.head(10)

,Postal Code,Borough,Neighbourhood,new_nghbr
0,M1A,Not assigned,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge","Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek","Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill","Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn,Woburn
5,M1H,Scarborough,Cedarbrae,Cedarbrae
6,M1J,Scarborough,Scarborough Village,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park","Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge","Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West","Cliffside, Cliffcrest, Scarborough Village West"


 use the .shape method to print the number of rows of your dataframe.

In [66]:
can_toronto_df = pd.DataFrame(toronto_df.groupby(['Postal Code','Borough'])['new_nghbr'].apply(','.join).reset_index())

# Renaming Columns to the same one in the image given:
can_toronto_df = can_toronto_df.rename(columns = {"Postal Code": "Postal Code","new_nghbr":"Neighborhood"})

In [67]:
print(can_toronto_df.head())
print("\n The Final Shape of the dataframe is  - ",can_toronto_df.shape)

  Postal Code       Borough                            Neighborhood
0         M1A  Not assigned                            Not assigned
1         M1B   Scarborough                          Malvern, Rouge
2         M1C   Scarborough  Rouge Hill, Port Union, Highland Creek
3         M1E   Scarborough       Guildwood, Morningside, West Hill
4         M1G   Scarborough                                  Woburn

 The Final Shape of the dataframe is  -  (180, 3)
